In [ ]:
# Cell 1: Imports and setup
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import colors
from matplotlib.widgets import Slider
import matplotlib
import matplotlib.font_manager

from medmnist import VesselMNIST3D
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Available devices:", tf.config.list_physical_devices())

In [ ]:
# Cell 2: Load data
train_dataset = VesselMNIST3D(split='train', size=28, download=True)
trainx = []
trainy = []

test_dataset = VesselMNIST3D(split='test', size=28, download=True)
testx = []
testy = []

val_dataset = VesselMNIST3D(split='val', size=28, download=True)
valx = []
valy = []

for i in range(len(train_dataset)):
    trainx.append(train_dataset[i][0])
    trainy.append(train_dataset[i][1])

for i in range(len(test_dataset)):
    testx.append(test_dataset[i][0])
    testy.append(test_dataset[i][1])

for i in range(len(val_dataset)):
    valx.append(val_dataset[i][0])
    valy.append(val_dataset[i][1])

print("Data loaded successfully!")
print(f"Training samples: {len(trainx)}")
print(f"Validation samples: {len(valx)}")
print(f"Test samples: {len(testx)}")

In [ ]:
# Cell 3: Analyze original class distribution
train_labels = np.array(trainy).flatten()
val_labels = np.array(valy).flatten()
test_labels = np.array(testy).flatten()

unique_train, counts_train = np.unique(train_labels, return_counts=True)
unique_val, counts_val = np.unique(val_labels, return_counts=True)
unique_test, counts_test = np.unique(test_labels, return_counts=True)

print(f"Training - Class 0 (Healthy): {counts_train[0]}, Class 1 (Aneurysm): {counts_train[1]}")
print(f"Validation - Class 0 (Healthy): {counts_val[0]}, Class 1 (Aneurysm): {counts_val[1]}")
print(f"Test - Class 0 (Healthy): {counts_test[0]}, Class 1 (Aneurysm): {counts_test[1]}")
print(f"\nClass imbalance ratio (train): {counts_train[1]/counts_train[0]:.3f}")

In [ ]:
# Cell 4: Augmentation functions and execution
import numpy as np
from scipy.ndimage import rotate, zoom, shift
from scipy.ndimage import gaussian_filter
import tensorflow as tf

def augment_3d_volume(volume, num_augmentations=5):
    augmented_volumes = []
    
    if hasattr(volume, 'numpy'):
        volume = volume.numpy()
    
    original_shape = volume.shape
    
    for _ in range(num_augmentations):
        aug_volume = volume.copy()
        
        if np.random.rand() > 0.5:
            angle = np.random.uniform(-15, 15)
            axes_options = [(1, 2), (1, 3), (2, 3)]
            axes = axes_options[np.random.randint(0, len(axes_options))]
            aug_volume = rotate(aug_volume, angle, axes=axes, reshape=False, mode='nearest')
        
        if np.random.rand() > 0.5:
            scale_factor = np.random.uniform(0.9, 1.1)
            zoom_factors = [1, scale_factor, scale_factor, scale_factor]
            aug_volume = zoom(aug_volume, zoom_factors, mode='nearest')
            aug_volume = resize_to_original(aug_volume, original_shape)
        
        if np.random.rand() > 0.5:
            axis = np.random.randint(1, 4)
            aug_volume = np.flip(aug_volume, axis=axis)
        
        if np.random.rand() > 0.5:
            shift_amount = [0] + [np.random.randint(-3, 4) for _ in range(3)]
            aug_volume = shift(aug_volume, shift_amount, mode='nearest')
        
        if np.random.rand() > 0.5:
            noise = np.random.normal(0, 0.01, aug_volume.shape)
            aug_volume = aug_volume + noise
            aug_volume = np.clip(aug_volume, 0, 1)
        
        if np.random.rand() > 0.5:
            brightness_factor = np.random.uniform(0.9, 1.1)
            aug_volume = aug_volume * brightness_factor
            aug_volume = np.clip(aug_volume, 0, 1)
        
        if np.random.rand() > 0.5:
            mean = np.mean(aug_volume)
            aug_volume = (aug_volume - mean) * np.random.uniform(0.9, 1.1) + mean
            aug_volume = np.clip(aug_volume, 0, 1)
        
        if np.random.rand() > 0.7:
            aug_volume = elastic_transform_3d(aug_volume, alpha=5, sigma=3)
        
        augmented_volumes.append(aug_volume)
    
    return augmented_volumes

def elastic_transform_3d(volume, alpha=10, sigma=3):
    shape = volume.shape[1:]
    
    dx = np.random.randn(*shape) * sigma
    dy = np.random.randn(*shape) * sigma
    dz = np.random.randn(*shape) * sigma
    
    dx = gaussian_filter(dx, sigma, mode='constant') * alpha
    dy = gaussian_filter(dy, sigma, mode='constant') * alpha
    dz = gaussian_filter(dz, sigma, mode='constant') * alpha
    
    z, y, x = np.meshgrid(
        np.arange(shape[0]),
        np.arange(shape[1]),
        np.arange(shape[2]),
        indexing='ij'
    )
    
    indices = [
        np.clip(z + dz, 0, shape[0] - 1).astype(int),
        np.clip(y + dy, 0, shape[1] - 1).astype(int),
        np.clip(x + dx, 0, shape[2] - 1).astype(int)
    ]
    
    result = np.zeros_like(volume)
    for c in range(volume.shape[0]):
        result[c] = volume[c][indices[0], indices[1], indices[2]]
    
    return result

def resize_to_original(volume, target_shape):
    current_shape = volume.shape
    result = volume.copy()
    
    for i in range(len(target_shape)):
        if current_shape[i] > target_shape[i]:
            diff = current_shape[i] - target_shape[i]
            start = diff // 2
            end = start + target_shape[i]
            result = np.take(result, range(start, end), axis=i)
        elif current_shape[i] < target_shape[i]:
            diff = target_shape[i] - current_shape[i]
            pad_before = diff // 2
            pad_after = diff - pad_before
            pad_width = [(0, 0)] * len(target_shape)
            pad_width[i] = (pad_before, pad_after)
            result = np.pad(result, pad_width, mode='edge')
    
    return result

print("Augmenting Class 1 (Aneurysm) samples...")

augmented_trainx = []
augmented_trainy = []

trainy_flat = [label.flatten()[0] if hasattr(label, 'flatten') else label for label in trainy]
class1_indices = [i for i, label in enumerate(trainy_flat) if label == 1]
print(f"Found {len(class1_indices)} Class 1 samples")

num_augmentations_per_sample = 6

print("Generating augmentations...")
for i, idx in enumerate(class1_indices):
    if i % 30 == 0:
        print(f"  Processing sample {i+1}/{len(class1_indices)}")
    
    original_volume = trainx[idx]
    augmented_volumes = augment_3d_volume(original_volume, num_augmentations_per_sample)
    
    for aug_vol in augmented_volumes:
        augmented_trainx.append(aug_vol)
        augmented_trainy.append(1)

print(f"Generated {len(augmented_trainx)} augmented samples for Class 1")

trainx_combined = trainx + augmented_trainx

trainy_combined_flat = []
for label in trainy:
    if isinstance(label, np.ndarray):
        trainy_combined_flat.append(label.flatten()[0])
    else:
        trainy_combined_flat.append(label)
        
trainy_combined_flat.extend(augmented_trainy)

trainx_tensor = tf.convert_to_tensor(trainx_combined, dtype=tf.float32)
trainy_tensor = tf.convert_to_tensor(trainy_combined_flat, dtype=tf.float32)
testx_tensor = tf.convert_to_tensor(testx, dtype=tf.float32)
testy_tensor = tf.convert_to_tensor(testy, dtype=tf.float32)
valx_tensor = tf.convert_to_tensor(valx, dtype=tf.float32)
valy_tensor = tf.convert_to_tensor(valy, dtype=tf.float32)

print(f"\nAugmented Training set shapes:")
print(f"  X: {trainx_tensor.shape}")
print(f"  y: {trainy_tensor.shape}")

train_labels_aug = np.array(trainy_combined_flat).flatten()
unique_aug, counts_aug = np.unique(train_labels_aug, return_counts=True)
print(f"\n{'='*60}")
print(f"Class Distribution Comparison:")
print(f"{'='*60}")
print(f"Original Training - Class 0: {counts_train[0]}, Class 1: {counts_train[1]}")
print(f"                    Ratio: {counts_train[1]/counts_train[0]:.3f}")
print(f"\nAugmented Training - Class 0: {counts_aug[0]}, Class 1: {counts_aug[1]}")
print(f"                     Ratio: {counts_aug[1]/counts_aug[0]:.3f}")
print(f"{'='*60}")

In [ ]:
# Cell 5: Data preprocessing
trainx_tensor = tf.transpose(trainx_tensor, [0, 2, 3, 4, 1])
valx_tensor = tf.transpose(valx_tensor, [0, 2, 3, 4, 1])
testx_tensor = tf.transpose(testx_tensor, [0, 2, 3, 4, 1])

print(f"After transpose - Training data shape: {trainx_tensor.shape}")

trainx_norm = trainx_tensor / 255.0
valx_norm = valx_tensor / 255.0
testx_norm = testx_tensor / 255.0

trainy_flat = tf.squeeze(trainy_tensor)
valy_flat = tf.squeeze(valy_tensor)
testy_flat = tf.squeeze(testy_tensor)

print(f"Normalized training data shape: {trainx_norm.shape}")
print(f"Training labels shape: {trainy_flat.shape}")
print(f"Data range: [{tf.reduce_min(trainx_norm):.3f}, {tf.reduce_max(trainx_norm):.3f}]")

train_labels_current = np.array(trainy_flat).flatten()
unique_current, counts_current = np.unique(train_labels_current, return_counts=True)
class_weight = {
    0: len(train_labels_current) / (2 * counts_current[0]),
    1: len(train_labels_current) / (2 * counts_current[1])
}
print(f"\nClass weights to handle imbalance: {class_weight}")

In [ ]:
# Cell 6: Build model
def build_3d_cnn(input_shape=(28, 28, 28, 1), dropout_rate=0.3):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv3D(32, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        layers.Dropout(dropout_rate),
        
        layers.Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        layers.Dropout(dropout_rate),
        
        layers.Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        layers.Dropout(dropout_rate),
        
        layers.GlobalAveragePooling3D(),
        
        layers.Dense(256, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='relu'),
        layers.Dropout(dropout_rate),
        
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

model = build_3d_cnn(input_shape=(28, 28, 28, 1), dropout_rate=0.3)
model.summary()

print("\nTotal parameters:", model.count_params())

In [ ]:
# Cell 7: Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

print("Model compiled with:")
print("- Optimizer: Adam (lr=0.001)")
print("- Loss: Binary Crossentropy")
print("- Metrics: Accuracy, AUC")

In [ ]:
# Cell 8: Setup callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=15,
    restore_best_weights=True,
    mode='max',
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

checkpoint = keras.callbacks.ModelCheckpoint(
    'best_vessel_model.h5',
    monitor='val_auc',
    save_best_only=True,
    mode='max',
    verbose=1
)

print("Callbacks configured:")
print("- Early Stopping: patience=15, monitor=val_auc")
print("- ReduceLROnPlateau: factor=0.5, patience=5")
print("- ModelCheckpoint: saves best val_auc")

In [ ]:
# Cell 9: Quick test (2 epochs)
history = model.fit(
    trainx_norm, trainy_flat,
    batch_size=16,
    epochs=2,
    validation_data=(valx_norm, valy_flat),
    class_weight=class_weight,
    verbose=1
)

print("\n" + "="*70)
print("INITIAL TEST RESULTS")
print("="*70)
print(f"Epoch 1 - Training Loss: {history.history['loss'][0]:.4f}, Val Loss: {history.history['val_loss'][0]:.4f}")
print(f"Epoch 2 - Training Loss: {history.history['loss'][1]:.4f}, Val Loss: {history.history['val_loss'][1]:.4f}")
print(f"\nTraining AUC: {history.history['auc'][-1]:.4f}")
print(f"Validation AUC: {history.history['val_auc'][-1]:.4f}")

if history.history['loss'][1] < history.history['loss'][0]:
    print("\nLoss is decreasing - model is learning!")
else:
    print("\nLoss not decreasing - may need to adjust hyperparameters")

In [ ]:
# Cell 10: Full training
print("Starting full training session...")

history_full = model.fit(
    trainx_norm, trainy_flat,
    batch_size=32,
    epochs=100,
    validation_data=(valx_norm, valy_flat),
    class_weight=class_weight,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(history_full.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history_full.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Model Loss Over Time')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(history_full.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history_full.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Accuracy Over Time')
axes[1].legend()
axes[1].grid(alpha=0.3)

axes[2].plot(history_full.history['auc'], label='Training AUC', linewidth=2)
axes[2].plot(history_full.history['val_auc'], label='Validation AUC', linewidth=2)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('AUC')
axes[2].set_title('Model AUC Over Time')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 11: Evaluation
print("\n" + "="*70)
print("MODEL EVALUATION ON TEST SET")
print("="*70)

model = keras.models.load_model('best_vessel_model.h5')

test_loss, test_acc, test_auc = model.evaluate(testx_norm, testy_flat, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

y_pred_proba = model.predict(testx_norm, verbose=0)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print("\nClassification Report:")
print(classification_report(test_labels, y_pred, 
                          target_names=['Healthy', 'Aneurysm'],
                          digits=4))

cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Healthy', 'Aneurysm'],
            yticklabels=['Healthy', 'Aneurysm'])
plt.title('Confusion Matrix - Test Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

fpr, tpr, thresholds = roc_curve(test_labels, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'Model (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Cell 12: Compare to baseline
print("VesselMNIST3D Baseline Performance (from MedMNIST paper):")
print("- ResNet18 (3D): AUC ~0.920, ACC ~0.890")
print("- Auto-sklearn: AUC ~0.917, ACC ~0.887")
print("\nYour Model Performance:")
print(f"- Test AUC: {test_auc:.4f}")
print(f"- Test ACC: {test_acc:.4f}")

if test_auc >= 0.920:
    print("\nModel matches or exceeds the baseline!")
elif test_auc >= 0.900:
    print("\nGood performance! Close to the baseline.")
else:
    print("\nRoom for improvement")